In [41]:
# just importing a random selection of things; we can fix this later
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegressionCV
%matplotlib inline

from sklearn.tree import DecisionTreeRegressor

from sklearn.model_selection import cross_val_score

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.grid_search import GridSearchCV

from pprint import pprint 

from sklearn.ensemble import BaggingClassifier
from sklearn import model_selection
from random import seed
from sklearn.model_selection import cross_val_predict

from sklearn.ensemble import RandomForestClassifier
from itertools import product
from collections import OrderedDict
from itertools import product

from sklearn import model_selection
from sklearn.svm import SVC
from sklearn import neighbors
from sklearn.ensemble import VotingClassifier

from sklearn.model_selection import train_test_split
from __future__ import print_function # For Python 2 / 3 compatability
from sklearn.metrics import confusion_matrix
import itertools

import warnings
from sklearn import preprocessing
%autosave 300
warnings.filterwarnings("ignore")

Autosaving every 300 seconds


<font color = magenta>
# 11.22.17 from here onward  - ADNIMERGE dataset

In [27]:
# load ADNIMERGE.csv, the main dataset (we will add our own variables to it). 
ADNI_merge = pd.read_csv('ADNIMERGE.csv')
ADNI_merge.head()

,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
0,2,011_S_0002,bl,11,ADNI1,ADNI1,2005-09-08,CN,74.3,Male,...,NaN,NaN,1.36926,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
1,3,011_S_0003,bl,11,ADNI1,ADNI1,2005-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.000000,0.00000,0,0,2017-08-13 23:50:48.0
2,3,011_S_0003,m06,11,ADNI1,ADNI1,2006-03-13,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.498289,5.96721,6,6,2017-08-13 23:50:48.0
3,3,011_S_0003,m12,11,ADNI1,ADNI1,2006-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,0.999316,11.96720,12,12,2017-08-13 23:50:48.0
4,3,011_S_0003,m24,11,ADNI1,ADNI1,2007-09-12,AD,81.3,Male,...,NaN,NaN,1.09079,NaN,NaN,1.998630,23.93440,24,24,2017-08-13 23:50:48.0


<font color = magenta> 
## NOTE: this is new code (11.23.17) because missing values are also denoted by "-4" and "-1". 
### Please see the training powerpoint for more info: https://adni.loni.usc.edu/wp-content/uploads/2012/08/slide_data_training_part2_reduced-size.pdf

In [28]:
''' ADNI2 data within ADNIMERGE.csv '''
# select only the rows that correspond to ADN2
ADNI2= ADNI_merge[ADNI_merge['COLPROT']=='ADNI2']
print(ADNI2.shape)
ADNI2.head()


(6937, 94)


,RID,PTID,VISCODE,SITE,COLPROT,ORIGPROT,EXAMDATE,DX_bl,AGE,PTGENDER,...,EcogSPDivatt_bl,EcogSPTotal_bl,FDG_bl,PIB_bl,AV45_bl,Years_bl,Month_bl,Month,M,update_stamp
58,5282,082_S_5282,bl,82,ADNI2,ADNI2,2013-09-09,SMC,66.9,Male,...,1.00000,1.25641,1.13549,NaN,1.326790,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
59,5280,100_S_5280,m24,100,ADNI2,ADNI2,2015-09-29,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,2.031490,24.32790,24,24,2017-10-06 23:20:00.0
68,5280,100_S_5280,m06,100,ADNI2,ADNI2,2014-03-06,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.465435,5.57377,6,6,2017-10-06 23:20:00.0
91,5280,100_S_5280,bl,100,ADNI2,ADNI2,2013-09-17,SMC,67.5,Male,...,1.00000,1.25000,1.25195,NaN,0.983143,0.000000,0.00000,0,0,2017-10-06 23:20:00.0
92,5279,082_S_5279,bl,82,ADNI2,ADNI2,2013-10-23,SMC,68.5,Male,...,1.33333,1.37838,1.50629,NaN,0.985156,0.000000,0.00000,0,0,2017-10-06 23:20:00.0


In [29]:
# It seems there were some 'unknown'..I replaced them with nan
import numpy as np
ADNI2 = ADNI2.replace('Unknown', np.nan)

In [30]:
# calculate the % of each column that is NaNs and print in descending order
nan_series = (ADNI2.isnull().sum()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(nan_series)

PIB                         1.000000
PIB_bl                      0.992936
FDG                         0.806833
AV45                        0.726251
FLDSTRENG                   0.553986
MidTemp                     0.543895
Fusiform                    0.543895
Entorhinal                  0.543895
Ventricles                  0.500793
Hippocampus                 0.497189
WholeBrain                  0.472106
FSVERSION                   0.441401
ICV                         0.441401
EcogSPOrgan                 0.363269
EcogPtOrgan                 0.356927
EcogSPDivatt                0.352314
MOCA                        0.350584
RAVLT_perc_forgetting       0.349575
EcogPtVisspat               0.347268
EcogPtDivatt                0.346836
EcogSPVisspat               0.346259
RAVLT_forgetting            0.345827
RAVLT_learning              0.344385
RAVLT_immediate             0.344385
EcogPtPlan                  0.342944
EcogSPPlan                  0.342655
EcogPtLang                  0.341790
A

In [31]:
# calculate the % of each column that is -1 and print in descending order
minus_one_series = (ADNI2[ADNI2 == -1].count()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(minus_one_series)

RAVLT_forgetting_bl         0.012830
RAVLT_learning              0.009658
RAVLT_learning_bl           0.008217
RAVLT_forgetting            0.007496
EcogSPTotal                 0.000000
EcogSPDivatt                0.000000
RAVLT_perc_forgetting       0.000000
FAQ                         0.000000
MOCA                        0.000000
EcogPtMem                   0.000000
EcogPtLang                  0.000000
EcogPtVisspat               0.000000
EcogPtPlan                  0.000000
EcogPtOrgan                 0.000000
FSVERSION                   0.000000
FLDSTRENG                   0.000000
Hippocampus                 0.000000
EcogPtTotal                 0.000000
EcogSPMem                   0.000000
EcogSPLang                  0.000000
EcogSPVisspat               0.000000
EcogSPPlan                  0.000000
EcogSPOrgan                 0.000000
EcogPtDivatt                0.000000
update_stamp                0.000000
RAVLT_immediate             0.000000
PTEDUCAT                    0.000000
P

In [32]:
# calculate the % of each column that is -4 and print in descending order
minus_one_series = (ADNI2[ADNI2 == -4].count()/len(ADNI2['DX_bl'])).sort_values(ascending=False)
print(minus_one_series)

RAVLT_forgetting            0.000288
RAVLT_learning              0.000288
RAVLT_forgetting_bl         0.000144
EcogPtDivatt                0.000000
RAVLT_perc_forgetting       0.000000
FAQ                         0.000000
MOCA                        0.000000
EcogPtMem                   0.000000
EcogPtLang                  0.000000
EcogPtVisspat               0.000000
EcogPtPlan                  0.000000
EcogPtOrgan                 0.000000
update_stamp                0.000000
EcogSPMem                   0.000000
EcogSPLang                  0.000000
EcogSPVisspat               0.000000
EcogSPPlan                  0.000000
EcogSPOrgan                 0.000000
EcogSPDivatt                0.000000
EcogSPTotal                 0.000000
FLDSTRENG                   0.000000
FSVERSION                   0.000000
EcogPtTotal                 0.000000
RAVLT_immediate             0.000000
Hippocampus                 0.000000
PTEDUCAT                    0.000000
PTID                        0.000000
V

In [33]:
## Drop the predictors that have more than 50% missing (thresh parameter)

ADNI2_clean= ADNI2.dropna(axis=1, how='any', thresh=len(ADNI2['DX_bl'])/2, inplace=False)
print("Shape of ADNI2 before dropping columns with NaN>50%")
print(ADNI2.shape)
print("\n")
print("Shape of ADNI2 after dropping columns with NaN>50%")
print(ADNI2_clean.shape)

Shape of ADNI2 before dropping columns with NaN>50%
(6937, 94)


Shape of ADNI2 after dropping columns with NaN>50%
(6937, 85)


In [34]:
# The percentage for -4 and -1 is really small, so in order to make things easier in the imputation, I just delete these
# observations
ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -4].index)
print("Shape of ADNI2 after dropping observations containing -4")
print(ADNI2_clean.shape)
print("\n")

ADNI2_clean = ADNI2_clean.drop(ADNI2_clean[ADNI2_clean.values == -1 ].index)
print("Shape of ADNI2 after dropping observations containing -1")
print(ADNI2_clean.shape)


Shape of ADNI2 after dropping observations containing -4
(6933, 85)


Shape of ADNI2 after dropping observations containing -1
(6690, 85)


In [35]:
#Check the unique values for each predictor. 
for col in ADNI2_clean.columns:
    print('unique values for', col , len(ADNI2_clean[col].value_counts()))

unique values for RID 1154
unique values for PTID 1154
unique values for VISCODE 23
unique values for SITE 58
unique values for COLPROT 1
unique values for ORIGPROT 3
unique values for EXAMDATE 1345
unique values for DX_bl 5
unique values for AGE 293
unique values for PTGENDER 2
unique values for PTEDUCAT 14
unique values for PTETHCAT 2
unique values for PTRACCAT 6
unique values for PTMARRY 4
unique values for APOE4 3
unique values for CDRSB 28
unique values for ADAS11 62
unique values for ADAS13 78
unique values for MMSE 29
unique values for RAVLT_immediate 76
unique values for RAVLT_learning 18
unique values for RAVLT_forgetting 20
unique values for RAVLT_perc_forgetting 82
unique values for FAQ 31
unique values for MOCA 31
unique values for EcogPtMem 57
unique values for EcogPtLang 68
unique values for EcogPtVisspat 51
unique values for EcogPtPlan 29
unique values for EcogPtOrgan 37
unique values for EcogPtDivatt 19
unique values for EcogPtTotal 474
unique values for EcogSPMem 57
un

Hi guys, <br>
I went through all the predictors. It took me a while to go through all the predictors but here below you see their description. <br>
Here below there is my strategy; I will discuss it with Hsiang and we will implement it <br>
** DOUBLE CLICK on it to see it**




                                                 **STRATEGY**

'RID',                    They should not be used for prediction, I'd keep them just for merging the prediction?
'PTID',                   They should not be used for prediction, I'd keep them just for merging the prediction?      
'VISCODE'                 They should not be used for prediction, I'd keep them just for merging the prediction?        'SITE'                    Shall we discard for the prediction?
'COLPROT'                 Shall we discard for the prediction?
'ORIGPROT'                Shall we discard for the prediction?
'EXAMDATE'                Shall we discard for the prediction?
'DX_bl'                   Hotone, categorical
'AGE',                    OK, int
'PTGENDER'                Hotone, categorical
'PTEDUCAT'                OK, int
'PTETHCAT',               Hotone, categorical
'PTRACCAT',               Hotone, categorical 
'PTMARRY'                 Hotone, categorical
'APOE4'                   Hotone, categorical   Class of protein
'CDRSB'                   OK,float              Clinical Dementia Rating Scale–Sum of Boxes
'ADAS11'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'ADAS13'                  OK,float              Alzheimer's Disease Assessment Scale-Cognitive Subscale
'MMSE',                   OK,float              Mini Mental State Examination
'RAVLT_immediate'         OK,float              Rey Auditory Verbal Learning Test
'RAVLT_learning'          OK,float              Rey Auditory Verbal Learning Test
'RAVLT_forgetting',       OK,float              Rey Auditory Verbal Learning Test
'RAVLT_perc_forgetting'   OK,float              Rey Auditory Verbal Learning Test
'FAQ'                     OK,float              Functional Assessment Questionnaire
'MOCA'                    OK,float              Montreal Cognitive Assessment (MoCA)
'EcogPtMem'               OK,float              Cognitive
'EcogPtLang',             OK,float              Cognitive
'EcogPtVisspat'           OK,float              Cognitive
'EcogPtPlan'              OK,float              Cognitive
'EcogPtOrgan'             OK,float              Cognitive
'EcogPtDivatt'            OK,float              Cognitive
'EcogPtTotal'             OK,float              Cognitive
'EcogSPMem'               OK,float              Cognitive
'EcogSPLang'              OK,float              Cognitive
'EcogSPVisspat'           OK,float              Cognitive
'EcogSPPlan',             OK,float              Cognitive
'EcogSPOrgan'             OK,float              Cognitive
'EcogSPDivatt'            OK,float              Cognitive
'EcogSPTotal'             OK,float              Cognitive
'FSVERSION',              Just one value, discard
'Hippocampus'             OK,float              Hippocampus volume
'WholeBrain'              OK,float              Brain volume
'ICV'                     OK,float              Intracranial volume
'DX'                      OUR Y, ENCODING
'EXAMDATE_bl'             Shall we discard for the prediction?
'CDRSB_bl'                       ***FROM HERE BASELINE***
'ADAS11_bl'
'ADAS13_bl'
'MMSE_bl'
'RAVLT_immediate_bl'
'RAVLT_learning_bl'
'RAVLT_forgetting_bl'
'RAVLT_perc_forgetting_bl',
'FAQ_bl', 'FLDSTRENG_bl'
'FSVERSION_bl'
'Ventricles_bl',
'Hippocampus_bl'
'WholeBrain_bl'
'Entorhinal_bl'
'Fusiform_bl',
'MidTemp_bl'
'ICV_bl'
'MOCA_bl'
'EcogPtMem_bl'
'EcogPtLang_bl',
'EcogPtVisspat_bl'
'EcogPtPlan_bl'
'EcogPtOrgan_bl',
'EcogPtDivatt_bl'
'EcogPtTotal_bl'
'EcogSPMem_bl'
'EcogSPLang_bl',
'EcogSPVisspat_bl'
'EcogSPPlan_bl'
'EcogSPOrgan_bl',
'EcogSPDivatt_bl'
'EcogSPTotal_bl'
'FDG_bl'                   OK,float               fluorescence connect to PET
'AV45_bl'                  OK,float               average survival
'Years_bl',                OK,float               years form baseline
'Month_bl'                 Shall we discard for the prediction? We already have 'Years_bl'
'Month'                    Shall we discard for the prediction? We already have 'Years_bl'
'M'                        Shall we discard for the prediction? We already have 'Years_bl'
'update_stamp              Shall we discard for the prediction? 

In [36]:
#Drop columns
print(ADNI2_clean.shape)
ADNI2_clean.drop (['M','Month', 'Month_bl','update_stamp',
                                'FSVERSION', 'EXAMDATE','ORIGPROT' ,
                                'COLPROT', 'SITE','ORIGPROT'], axis =1, inplace= True)

(6690, 85)


In [37]:
ADNI2_clean['PTMARRY'].unique()

array(['Married', 'Never married', 'Widowed', 'Divorced', nan], dtype=object)

In [38]:
#Hot one encoding
categorical_columns_g2cols = ['DX_bl', 'PTGENDER', 'PTETHCAT','PTRACCAT','PTMARRY','APOE4']                                 
ADNI2_clean = pd.get_dummies(ADNI2_clean, columns=categorical_columns_g2cols, prefix = categorical_columns_g2cols , drop_first=True)

In [39]:
print(ADNI2_clean.columns)

Index(['RID', 'PTID', 'VISCODE', 'AGE', 'PTEDUCAT', 'CDRSB', 'ADAS11',
       'ADAS13', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning',
       'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'EcogPtMem',
       'EcogPtLang', 'EcogPtVisspat', 'EcogPtPlan', 'EcogPtOrgan',
       'EcogPtDivatt', 'EcogPtTotal', 'EcogSPMem', 'EcogSPLang',
       'EcogSPVisspat', 'EcogSPPlan', 'EcogSPOrgan', 'EcogSPDivatt',
       'EcogSPTotal', 'Hippocampus', 'WholeBrain', 'ICV', 'DX', 'EXAMDATE_bl',
       'CDRSB_bl', 'ADAS11_bl', 'ADAS13_bl', 'MMSE_bl', 'RAVLT_immediate_bl',
       'RAVLT_learning_bl', 'RAVLT_forgetting_bl', 'RAVLT_perc_forgetting_bl',
       'FAQ_bl', 'FLDSTRENG_bl', 'FSVERSION_bl', 'Ventricles_bl',
       'Hippocampus_bl', 'WholeBrain_bl', 'Entorhinal_bl', 'Fusiform_bl',
       'MidTemp_bl', 'ICV_bl', 'MOCA_bl', 'EcogPtMem_bl', 'EcogPtLang_bl',
       'EcogPtVisspat_bl', 'EcogPtPlan_bl', 'EcogPtOrgan_bl',
       'EcogPtDivatt_bl', 'EcogPtTotal_bl', 'EcogSPMem_bl', 'EcogSPLang_b

In [120]:
#Function to split the data in train and test
def split (data, percentage):
    np.random.seed(9001) 
    msk = np.random.rand(len(data)) < percentage
    train = data[msk]
    test = data[~msk]
    return train, test



In [122]:
# Imputation by mean

In [184]:
# Imputation by model
